# ch06. 데이터가 추가된 RNN
RNN: 성능이 좋지 X
이유) 시계열 데이터에서 시간적으로 멀리 떨어진, _장기 의존 관계를 잘 학습할 수 없음_

=> 해결책: LSTM, GRU 와 같은 **게이트가 추가된 RNN** 을 많이 사용함

## 6.1 RNN의 문제점
: 시계열 데이터의 장기 의존 관계를 학습하기 어려움
원인) BPTT에서 기울기 소실 or 기울기 폭발이 발생하기 때문

### 6.1.1 RNN 복습
RNN: 순환 경로를 가지고 있음.
![](img/6-1.png)
- 시계열 데이터인 x_t를 입력하면 h_t 출력
- h_t: RNN 계층의 은닉 상태 (과거 정보 저장)

=> 특징: 이전 시각의 은닉 상태를 이용함
=> 과거 정보 계승 가능

<RNN계층의 순전파에서 수행하는 연산>
- 행렬의 곱 (MatMul)
- 합 (+)
- 활성화함수 tanh()


### 6.1.2 기울기 소실 또는 폭발
언어 모델: 주어진 단어들을 기초로 다음에 출현할 단어를 예측하는 일

![](img/6-4.png)
=> RNN 계층이 과거 방향으로 **'의미있는 기울기'** 를 전달함으로써 시간 방향의 의존 관계를 학습할 수 있음
- 기울기: (원래대로라면) 학습해야 할 의미가 있는 정보 존재
- 이것을 과거로 전달하면서 장기의존 관계 학습
- but, 기울기가 중간에 사그라들면 가중치 매개변수는 갱신 X
- ∴ 장기의존 관계를 학습 X (기본 RNN: 기울기 소실 or 기울기 폭발의 운명)

### 6.1.3 기울기 소실과 폭발의 원인
![](img/6-5.png)

길이가 T인 시계열 데이터 -> T번째 정답 레이블로부터 전해지는 기울기의 변화에 주목
- 시간 방향 기울기
    - 역전파로 전해지는 기울기: 'tanh' -> '+' -> 'MatMul'
        - '+' : 상류 그대로 하류 전달
        - 'tanh': tanh 함수를 T번 통과하면 기울기도 T번 반복해서 감소
            ![](img/6-6.png)
                - RNN 계층에서 tanh 대신 ReLU사용-> 기울기 소실 개선 가능
        - 'MatMul': Wh가 1보다 크면 지수적으로 증가, 1보다 작으면 지수적으로 감소

### 6.1.4 기울기 폭발 대책
: 기울기 클리핑


In [1]:
import numpy as np

dW1 = np.random.randn(3,3) * 10
dW2 = np.random.randn(3,3) * 10
grads = [dW1, dW2]
max_norm = 5.0

def clip_grads(grads, max_norm):
    total_norm = 0
    for grad in grads:
        total_norm += np.sum(grad ** 2)
    total_norm = np.sqrt(total_norm)

    rate = max_norm / total_norm
    if rate < 1:
        for grad in grads:
            grad *= rate

## 6.2 기울기 소실과 LSTM
'게이트가 추가된 RNN': ex. LSTM, GRU

### 6.2.1 LSTM의 인터페이스
![](img/6-11.png)

- LSTM의 인터페이스: c라는 경로 존재
- c: 기억 셀 -> LSTM 전용의 기억 매커니즘
    - 특징: 데이터를 자기 자신으로만 주고받음
    - LSTM 계층 내에서만 완결, 다른 계층으로 출력 X
- h: LSTM의 은닉 상태
    - RNN계층과 마찬가지로 다른 계층으로 (위쪽으로) 출력됨

### 6.2.2 LSTM 계층 조립하기
![](img/6-12.png)
- c_t: 시간t에서의 LSTM의 기억이 저장되어 있음.
    - 과거로부터 시각 t까지에 필요한 모든 정보 저장
    - 필요한 정보를 모두 간직한 이 기억을 바탕으로, 외부 계층 (or 다음 시각의 LSTM에) 은닉상태 h_t를 출력함
    - 출력하는 h_t: 기억 셀의 값을 tanh함수로 변환한 값

=> c_t는 3개의 입력 (c_t-1, h_t-1, x_t)로부터 '어떤 계산'을 수행한 값

`게이트`: 데이터의 흐름 제어
- LSTM에서 사용하는 게이트 : 열기/닫기, 물의 양 제어 가능
- '게이트를 얼마나 열까'도 데이터로부터 자동으로 학습

### 6.2.3 output 게이트
[output 게이트]
: 다음 은닉상태 h_t의 출력을 담당하는 게이트
- tanh(c_t)의 각 원소에 대해 '그것이 다음 시각의 은닉 상태에 얼마나 중요한가'를 조정

- 열림 상태: x_t와 h_t-1로부터 구함
- h_t = o ◉ tanh(c_t)
    - 아다마르 곱: 원소별 곱


### 6.2.4 forget 게이트
[forget 게이트]
: c_t-1의 기억 중에서 불필요한 기억을 잊게 해주는 게이트
![](img/math6-3.png)

: forget 게이트의 출력

### 6.2.5 새로운 기억 셀 (g)
tanh노드가 계산한 결과가 이전 시각의 기억 셀 c_t-1에 더해짐
=> 기억 셀에 새로운 '정보'가 추가된 것
![](img/math6-4.png)

이 g가 이전 시각의 기억 셀인 c_t-1에 더해짐으로써 새로운 기억 생성


### 6.2.6 input 게이트
: g의 각 원소가 새로 추가되는 정보로써의 가치가 얼마나 큰지를 판단함.

![](img/math6-5.png)
i와 g의 원소별 곱 결과를 기억 셀에 추가

--
LSTM의 전체적인 구조
![](img/6-18.png)


### 6.2.7 LSTM의 기울기 흐름
하지만, 어떻게 기울기 소실을 없애주는 걸까?
_기억셀 c의 역전파_ 주목!

![](img/6-19.png)
- 기억 셀 c의 역전파 : '+' 와 'x' 노드만 지나게
    - '+' : 상류 기울기 그대로 하류
        - 기울기 변화(감소) x
    - 'x': '행렬 곱'이 아닌 '원소별 곱'을 계산
        - 매 시각 다른 게이트 값을 이용해 원소별 곱 계산
        - => 곱셈의 효과 누적 X
        - 기울기 소실 X
        - forget node가 제어함.

## 6.3 LSTM 구현
